In [1]:
import torch

In [3]:
a = torch.randn(2, 5, 4)
print(a)

tensor([[[ 0.6022, -1.0035,  0.0287, -0.4071],
         [-1.9933, -1.3943,  2.0388,  1.8566],
         [ 0.3161,  0.8706,  1.2285,  0.5301],
         [-0.5484,  1.0004, -1.0569, -0.4895],
         [-0.8551, -0.0047, -0.7638,  1.1068]],

        [[ 1.2396, -0.3259, -0.7993,  0.2577],
         [-0.9632, -0.7329,  1.0872, -0.5243],
         [-0.3631, -1.9192, -0.5292, -0.4602],
         [-0.8766,  1.1034, -1.5836, -0.2817],
         [ 0.1851, -1.2992, -1.1487, -0.5477]]])


In [4]:
b = torch.argmax(a, dim=-1)
print(b)

tensor([[0, 2, 2, 1, 3],
        [0, 2, 0, 1, 0]])


In [5]:
print(b.shape)

torch.Size([2, 5])


In [7]:
c = torch.randint(0, 5, (2, 5))
print(c)


tensor([[3, 1, 4, 0, 1],
        [0, 0, 3, 4, 4]])


In [9]:
from ignite.metrics.nlp import Bleu
m = Bleu(ngram=4, smooth="smooth1")
m.update((b, c))
print(m.compute())

tensor(0., dtype=torch.float64)


In [11]:
d = torch.tensor(
    [[3, 1, 4, 0, 1],
    [0, 0, 3, 4, 4]],
    dtype=torch.long
)

In [12]:
m1 = Bleu(ngram=4, smooth="smooth1")
m1.update((c, d))
print(m1.compute())

tensor(0., dtype=torch.float64)


In [13]:
print(c)

tensor([[3, 1, 4, 0, 1],
        [0, 0, 3, 4, 4]])


In [14]:
print(d)

tensor([[3, 1, 4, 0, 1],
        [0, 0, 3, 4, 4]])


In [17]:
m = Bleu(ngram=4, smooth="smooth1")

y_pred = "the the the the the the the"
y = "the cat is on the mat"

m.update((y_pred.split(), [y.split()]))

print(m.compute())

tensor(0.0393, dtype=torch.float64)


In [1]:
import torch 
from torch_geometric.data import Data, DataLoader

In [2]:
x1 = torch.randn(5, 16)
edge1 = torch.tensor([[0, 0, 0, 0], [1, 2, 3, 4]])
attr1 = torch.rand(4)
x2 = torch.rand(4, 16)
edge2 = torch.tensor([[0, 0 ,0], [1, 2, 3]])
attr2 = torch.rand(3)
data1 = Data(x=x1, edge_index=edge1, edge_attr=attr1)
data2 = Data(x=x2, edge_index=edge2, edge_attr=attr2)
data = [data1, data2]

In [3]:
loader = DataLoader(data, batch_size=2)

In [4]:
batch = next(iter(loader))
batch

Batch(batch=[9], edge_attr=[7], edge_index=[2, 7], ptr=[3], x=[9, 16])

In [5]:
batch.batch

tensor([0, 0, 0, 0, 0, 1, 1, 1, 1])

In [6]:
batch.edge_index

tensor([[0, 0, 0, 0, 5, 5, 5],
        [1, 2, 3, 4, 6, 7, 8]])

In [7]:
batch.ptr

tensor([0, 5, 9])

In [8]:
divide_node_num = 2

In [9]:
def get_subgraph_info_from_batch(batch):
    comp = 0
    pos = 0
    graph_pos_in_batch = [0] # record begin positions and end positions of every subgraph
    graph_length = [] # use a list to store the node nums in subgraph
    for i in range(len(batch)):
        if batch[i] != comp:
            graph_pos_in_batch.append(i)
            graph_length.append(i-pos)
            comp = batch[i]
            pos = i
    graph_length.append(len(batch)-pos)
    return graph_pos_in_batch, graph_length

In [10]:
def partition_graph(x, edge_index, edge_attr, batch):        
    nodes_list = [] # record all nodes number for each subgraph in total batch
    graph_pos_in_batch, graph_length = get_subgraph_info_from_batch(batch)
    max_seq_len = max(graph_length)
    subgraph_num = int(max_seq_len/divide_node_num) + 1
    for i in range(subgraph_num):
        nodes = []
        for j in range(len(graph_pos_in_batch)):
            if graph_length[j] > i * divide_node_num:
                if graph_length[j] > (i+1) * divide_node_num:
                    subgraph_len = divide_node_num
                else:
                    subgraph_len = graph_length[j] - i * divide_node_num   
                for m in range(subgraph_len):
                    nodes.append(graph_pos_in_batch[j] + m)          
        nodes_list.append(set(nodes)) 
    # only count the edge whose target node in subgraph
    sub_edge_src = [[] for _ in range(subgraph_num)]
    sub_edge_tgt = [[] for _ in range(subgraph_num)]
    sub_edge_attr = [[] for _ in range(subgraph_num)]
    # print('nodes_list', nodes_list)
    node_num = len(x)
    node_subgraph_index = [0 for _ in range(node_num)] # use a list to store the subgraph numbers for all nodes
    for i in range(len(nodes_list)):
        for node in nodes_list[i]:
            node_subgraph_index[node] = i

    for i in range(len(edge_index[1])):
        src = edge_index[0][i].item()
        tgt = edge_index[1][i].item()
        sub_edge_src[node_subgraph_index[tgt]].append(src)
        sub_edge_tgt[node_subgraph_index[tgt]].append(tgt)
        sub_edge_attr[node_subgraph_index[tgt]].append(edge_attr[i].item())
    edge_index_list = []
    edge_attr_list = []
    for i in range(subgraph_num):
        edge_index_list.append(torch.tensor([sub_edge_src[i], sub_edge_tgt[i]], dtype=torch.long))
        edge_attr_list.append(torch.tensor(sub_edge_attr[i], dtype=torch.long))
    return edge_index_list, edge_attr_list

In [11]:
edge_index_list, edge_attr_list = partition_graph(batch.x, batch.edge_index, batch.edge_attr, batch.batch)

In [12]:
edge_index_list

[tensor([[0, 0, 0, 5, 5],
         [2, 3, 4, 7, 8]]),
 tensor([[0, 5],
         [1, 6]]),
 tensor([], size=(2, 0), dtype=torch.int64)]

In [13]:
edge_attr_list

[tensor([0, 0, 0, 0, 0]), tensor([0, 0]), tensor([], dtype=torch.int64)]